In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

from keras_preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from keras.applications.vgg19 import VGG19
from sklearn.metrics import *
from pathlib import Path

In [2]:
input_dir = Path('../input')

siim_i_dir = os.path.join(input_dir, 'siim-i')
image_dir = os.path.join(input_dir, 'siim-isic-melanoma-classification/jpeg')

train_image_dir = os.path.join(image_dir, 'train')

In [3]:
train_file = os.path.join(siim_i_dir, 'train_data.csv')
valid_file = os.path.join(siim_i_dir, 'valid_data.csv')

train_df = pd.read_csv(train_file, index_col=0)
valid_df = pd.read_csv(train_file, index_col=0)

In [4]:
train_df_copy = train_df.copy()
valid_df_copy = valid_df.copy()

In [5]:
def format_image_path(image_name):
    
    return image_name + '.jpg'

In [6]:
train_df_copy['image_path'] = train_df_copy.image_name.map(format_image_path)
valid_df_copy['image_path'] = valid_df_copy.image_name.map(format_image_path)

In [7]:
train_df_copy['target'] = train_df_copy['target'].astype(str)
valid_df_copy['target'] = valid_df_copy['target'].astype(str)

In [8]:
batch_size = 128
image_size = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255.,  
    horizontal_flip = True, 
    vertical_flip = True, 
    height_shift_range= 0.1, 
    width_shift_range=0.1, 
    rotation_range=90,
    validation_split=0.2
)

In [9]:
train_generator=train_datagen.flow_from_dataframe(
    dataframe=train_df_copy,
    directory=train_image_dir,
    x_col="image_path",
    y_col='target', 
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=image_size,
    color_mode='rgb',
    subset='training'
)

valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train_df_copy,
    directory=train_image_dir,
    x_col="image_path",
    y_col='target', 
    batch_size=batch_size,
    seed=42,
    shuffle=False,
    class_mode='binary',
    target_size=image_size,
    color_mode='rgb',
    subset='validation'
)

Found 40904 validated image filenames belonging to 2 classes.
Found 10226 validated image filenames belonging to 2 classes.


In [10]:
num_classes = 1

IMG_WIDTH, IMG_HEIGHT = image_size

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

base_model.trainable = False

model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2156, activation = 'relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1280, activation = 'relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(540, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation = 'sigmoid')
])

model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 2156)              54091884  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2156)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1280)              2760960   
_________________________________________________________

In [11]:
from tensorflow import keras

model = keras.models.load_model('../input/modelv1/model.h5')

In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
EPOCHS = 100

model_file = os.path.join('model.h5')

early_stopping = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', 
                    patience=5,
                    restore_best_weights=True
                 )

save_best = tf.keras.callbacks.ModelCheckpoint(
                model_file, 
                monitor='val_accuracy', 
                save_best_only=True
            )

history = model.fit(
    train_generator, 
    epochs = EPOCHS, 
    validation_data=valid_generator, 
    callbacks=[early_stopping, save_best]
)

In [ ]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

training_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs_range=range(len(training_accuracy))

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, training_loss, label='Training Loss')
plt.plot(epochs_range, validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()